In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time


# Import API key
from pprint import pprint
from config import api_key 
from pandas.io.json import json_normalize

import urllib3
urllib3.disable_warnings()


In [3]:
#starting URL

keywords ="Florida Man"
url = "https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=" + api_key + "&keyword=Florida%20Man%5C&radius=300"

print(url)

https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=ceb89258ec1bc5b95870cba7d41ddfeba99e8bc7&keyword=Florida%20Man%5C&radius=300


In [4]:

titles_1 = []
summary_1 = []
link_1 = []
time_1 = []
source_1 = []
address_1 = []

articles = requests.get(url,verify=False).json()
print(articles['next'])
i = 0
while True:
   
    for result in articles['results'] :
        titles_1.append(result['title'])
        summary_1.append(result['summary'])
        time_1.append(result['time'])
        link_1.append(result['link'])
        source_1.append(result['source']['name'])
        address_1.append(result['source']['address'])
        i = i + 1
    if i >100000: 
        break
    articles = requests.get(articles['next'],verify=False).json()
        
    if articles == {}:
        break
            
    print(articles['next'])
    

https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=ceb89258ec1bc5b95870cba7d41ddfeba99e8bc7&keyword=Florida+Man%5C&radius=300&cursor=AoIIQ9pXqDluZXdzZmVlZC5hcnRpY2xlLjc5MzEzNjQ1
https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=ceb89258ec1bc5b95870cba7d41ddfeba99e8bc7&keyword=Florida+Man%5C&radius=300&cursor=AoIIQ9oIdzluZXdzZmVlZC5hcnRpY2xlLjc5MTI5Nzcw
https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=ceb89258ec1bc5b95870cba7d41ddfeba99e8bc7&keyword=Florida+Man%5C&radius=300&cursor=AoIIQ9nTsDluZXdzZmVlZC5hcnRpY2xlLjc5NTAzMTMy
https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=ceb89258ec1bc5b95870cba7d41ddfeba99e8bc7&keyword=Florida+Man%5C&radius=300&cursor=AoIIQ9msgzluZXdzZmVlZC5hcnRpY2xlLjc5NDI1NjIy
https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=ceb89258ec1bc5b95870cba7d41ddfeba99e8bc7&keyword=Florida+Man%5C&radius=300&cursor=AoIIQ9mTXDluZXdzZmVlZC5hcnRpY2

https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=ceb89258ec1bc5b95870cba7d41ddfeba99e8bc7&keyword=Florida+Man%5C&radius=300&cursor=AoIIQ9W4GzluZXdzZmVlZC5hcnRpY2xlLjc2MDIyMjg3
https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=ceb89258ec1bc5b95870cba7d41ddfeba99e8bc7&keyword=Florida+Man%5C&radius=300&cursor=AoIIQ9WeGTluZXdzZmVlZC5hcnRpY2xlLjc2MDgzNzc0
https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=ceb89258ec1bc5b95870cba7d41ddfeba99e8bc7&keyword=Florida+Man%5C&radius=300&cursor=AoIIQ9V9ZjluZXdzZmVlZC5hcnRpY2xlLjc2MDI3NzMw
https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=ceb89258ec1bc5b95870cba7d41ddfeba99e8bc7&keyword=Florida+Man%5C&radius=300&cursor=AoIIQ9VnSzluZXdzZmVlZC5hcnRpY2xlLjc2MTc2NjA0
https://www.accunewsapp.com/api/v1/articles/?lon=28.9025&lat=82.5926&apiKey=ceb89258ec1bc5b95870cba7d41ddfeba99e8bc7&keyword=Florida+Man%5C&radius=300&cursor=AoIIQ9VTgzluZXdzZmVlZC5hcnRpY2

KeyError: 'next'

In [6]:
news_df_dict = {'Title': titles_1,
          'Summary': summary_1,
          'Link': link_1,
          'Time': time_1,
          'Source': source_1,
          'Address': address_1}

news_df = pd.DataFrame(news_df_dict)

news_df.to_csv("Local_news_data.csv", index=False)